In [5]:
from tensorflow import keras

2023-11-02 04:40:39.215537: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 04:40:39.245241: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 04:40:39.245761: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 04:40:39.824582: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
def build_net_status(count: int, frame_shape: tuple = (8, 10, 2048)) -> tuple[list, list]:
    frame_inputs = []
    frame_outputs = []
    for i in range(count):
        frame_input = keras.layers.Input(shape=frame_shape, name=f'frame_{i}')
        frame_flatten = keras.layers.Flatten(name=f'frame_{i}_flatten')(frame_input)
        frame_output = keras.layers.Dense(10, name=f'frame_{i}_zip')(frame_flatten)
        frame_inputs.append(frame_input)
        frame_outputs.append(frame_output)

    return frame_inputs, frame_outputs


def build_net() -> keras.models.Model:
    frame_inputs, frame_outputs = build_net_status(4)
    # aqui concatenamos todas las capas de frames
    general_layer_frames = keras.layers.Concatenate(name='frames_concatenate')(frame_outputs)
    # ahora vamos a comprimir todos los frames en 20 neuronas
    general_layer_frames_zip = keras.layers.Dense(20, name='frames_zip')(general_layer_frames)
    # entrada de las acciones
    actions_input = keras.layers.Input(shape=(10,), name='actions_input')
    # ahora vamos a comprimir el vector de las acciones en 20 neuronas taqmbien para que este equilibrado con los frames
    actions_zip = keras.layers.Dense(20, name='actions_zip')(actions_input)
    # ahora concatenamos los estados y las acciones
    action_status = keras.layers.Concatenate(name='concatenate_action_status')([general_layer_frames_zip, actions_zip])
    # la salida general seria 1 neurona con el valor Q
    general_layer_output = keras.layers.Dense(1, name='q')(action_status)

    return keras.models.Model(
        name='MY_Q_NET',
        inputs=[*frame_inputs, actions_input],
        outputs=[general_layer_output],
    )

In [12]:
general_net: keras.models.Model = build_net()
general_net.compile(optimizer='adam', loss='mean_squared_error')
general_net.summary()

Model: "MY_Q_NET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 frame_0 (InputLayer)           [(None, 8, 10, 2048  0           []                               
                                )]                                                                
                                                                                                  
 frame_1 (InputLayer)           [(None, 8, 10, 2048  0           []                               
                                )]                                                                
                                                                                                  
 frame_2 (InputLayer)           [(None, 8, 10, 2048  0           []                               
                                )]                                                         